Description of the building
notes:
Pour les surfaces des murs, les premiers calculs ont été fait en considérant que les longueures correspondent aux longueurs exterieurs.  

In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


H = 3               # m hauteur des murs
L1= 10              # m longueur horizontale
L2= 8               # m largeur  du haut
L3= 6               # m largeur du bas
S1 = L1*H          # m² surface 1 intermédiaire
S2 = L2*H           # m² surface 2 intermédiaire
S3 = L3*H           # m² surface 3 intermédiaire
Smur1 = 2*S1+2*S3   # m² surface du mur du bas
Svitre = 4          # m² surface d'une vitre
Smur2= S1+S2 -Svitre        # m² surface d'un des murs du haut ou on enlèvre la surface des vitres
Smur3= S1             # m² surface du mur intérieur horizontale
Smur4= S2             # m² surface du mur intérieur verticale


50

Definition of thermo-physicals properties (we have kept the same value as the course) 

In [79]:
air = {'Density': 1.2,                      # kg/m³
       'Specific heat': 1000}               # J/(kg·K)

concrete = {'Conductivity': 1.400,          # W/(m·K)
            'Density': 2300.0,              # kg/m³
            'Specific heat': 880,           # J/(kg⋅K)
            'Width': 0.2}            # m²

insulation = {'Conductivity': 0.027,        # W/(m·K)
              'Density': 55.0,              # kg/m³
              'Specific heat': 1210,        # J/(kg⋅K)
              'Width': 0.08}          # m²

glass = {'Conductivity': 1.4,               # W/(m·K)
         'Density': 2500,                   # kg/m³
         'Specific heat': 1210,             # J/(kg⋅K)
         'Width': 0.04}                   # m²

layer = pd.DataFrame.from_dict({'Layer_out': concrete,
                               'Layer_in': insulation,
                               'Glass': glass},
                              orient='index')

# radiative properties
ε_wLW = 0.85    # long wave emmisivity: wall surface (concrete)
ε_gLW = 0.90    # long wave emmisivity: glass pyrex
α_wSW = 0.25    # short wave absortivity: white smooth surface
α_gSW = 0.38    # short wave absortivity: reflective blue glass
τ_gSW = 0.30    # short wave transmitance: reflective blue glass

σ = 5.67e-8     # W/(m²⋅K⁴) Stefan-Bolzmann constant
h=[8,25]
wall_width_interior=insulation['Width']
wall_width_exterior=insulation['Width']+concrete['Width']
pd.DataFrame([{'in': 8., 'out': 25}], index=['h'])  # W/(m²⋅K)

,in,out
h,8.0,25


Computation of thermal conductance of conduction

In [80]:
G_layer = layer['Conductivity'] / layer['Width'] #conductance par m²
pd.DataFrame(G_layer, columns=['Conductance'])
G_cd={'vitre':G_layer[2]*Svitre,
        'mur bas béton':Smur1*G_layer[0],
        'mur bas isolant':Smur1*G_layer[1],
        'mur haut isolant':Smur2*G_layer[1],
        'mur haut béton':Smur2*G_layer[0],
        'mur intérieur horizontal':Smur3*G_layer[0],
        'mur intérieur vertical':Smur4*G_layer[0]
}
pd.DataFrame.from_dict({'Conduction':G_cd})



,Conduction
mur bas béton,672.000
mur bas isolant,32.400
mur haut béton,350.000
mur haut isolant,16.875
mur intérieur horizontal,210.000
mur intérieur vertical,168.000
vitre,140.000


Computation of thermal conductance of convection

In [81]:
G_conv={'vitre interieur':h[0]*Svitre,
        'vitre interieur':h[1]*Svitre,
        'mur bas intérieur':Smur1*h[0],
        'mur bas exterieur':Smur1*h[1],
        'mur haut interieur':Smur2*h[1],
        'mur haut exterieur':Smur2*h[0],
        'mur intérieur horizontal':Smur3*h[0],
        'mur intérieur vertical':Smur4*h[0]
}
pd.DataFrame.from_dict({'Conduction':G_conv})

,Conduction
mur bas exterieur,2400
mur bas intérieur,768
mur haut exterieur,400
mur haut interieur,1250
mur intérieur horizontal,240
mur intérieur vertical,192
vitre interieur,100


Computation of thermal capacities

In [84]:
C_layer = layer['Density'] * layer['Specific heat']  * layer['Width']
pd.DataFrame(C_layer, columns=['Capacity'])
C_thermal = {'vitre':C_layer[2]*Svitre,
        'mur bas béton':Smur1*C_layer[0],
        'mur bas isolant':Smur1*C_layer[1],
        'mur haut isolant':Smur2*C_layer[1],
        'mur haut béton':Smur2*C_layer[0],
        'mur intérieur horizontal':Smur3*C_layer[0],
        'mur intérieur vertical':Smur4*C_layer[0]
}
pd.DataFrame.from_dict({'Thermal capacity':C_thermal})

,Thermal capacity
mur bas béton,38860800.0
mur bas isolant,511104.0
mur haut béton,20240000.0
mur haut isolant,266200.0
mur intérieur horizontal,12144000.0
mur intérieur vertical,9715200.0
vitre,484000.0


Computation of thermal conductance of advection
Hypothesis: windows closed, front bottom door open, lateral doors open, middle top door (closed or open)

In [83]:
Va1 = (L1-wall_width_exterior)*(L2-wall_width_exterior)*H    # m³, volume d'air de la pièce du haut
Va2 = (2*L1-wall_width_exterior)*(L2-wall_width_exterior)*H           # m³, volume d'air de la pièce du haut
ACH1 = 1        # 1/h, changement d'air pièce du bas
ACH2 = 0.5      # 1/h, changement d'air pièce du haut
G_adv = {   'Advection bas': air['Density']*air['Specific heat']*ACH1*Va1/3600,
            'Advection haut': air['Density']*air['Specific heat']*ACH2*Va2/3600,
            'Advection entre haut':10
}
pd.DataFrame.from_dict({'Conduction':G_adv})

,Conduction
Advection bas,75.0384
Advection entre haut,10.0000
Advection haut,76.1192
